Assignment:
    
AI Travel Agent & Expense Planner(Purpose: Trip planning for any city worldwide with Realtime data.")

• Real-time weather information
• Top attractions and activities
• Hotel cost calculation (per day × total days)
• Currency conversion to user's native currency
• Complete itinerary generation
• Total expense calculation
• generate a summary of the entire output

user_input
  |
search attraction and activity
1. search attracation
2. search restaurant
3. search activity
4. search transportation
  |
search weather forcasting
1. get current weather
2. get weather forcast
  |
search hotel costs
1. search hotel
2. estimate the hotel cost
3. budget_range
  |
calculate total cost
1. add
2. multiply
3. calculated total cost
4. calcualte the daily budget
    | 
currency_converion
1. get exchnage rate
2. convert currancy
    | 
Itinery generation
1. get day plan
2. crete full itinery
    |
create Trip Summary
    |
Retun complete traval plan

Note: if you know the OOPS then design this entire system using object and class in modular fashion.


deadline is till next friday 9PM IST


 everyone you can submit the assignments in this form. MAke sure to have one GitHub link and put all the assignments there https://forms.gle/g8RZ4qx8yvNcih4B7    
    

In [22]:
import operator
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,END, MessagesState, END, START
from dotenv import load_dotenv
from langchain.output_parsers import PydanticOutputParser
import os
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_tavily import TavilySearch
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import InMemorySaver

from langchain_community.tools import DuckDuckGoSearchRun


load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    api_key=OPENAI_API_KEY
)

search=DuckDuckGoSearchRun()

# First Workflow Itinerary Search and Summary

In [ ]:
import time

class State(TypedDict):
    destiny: str
    start_date: str
    end_date: str
    attractions: str
    restaurants: str
    activities: str
    weather: str
    hotels: str
    final_itinerary: str

def itinerary_tool(state:State):
    destiny = state["destiny"]
    start_date = state["start_date"]
    end_date = state["end_date"]

    print("Searching for Attractions...")
    best_att = search.invoke(f"Best Attractions in {destiny}")
    time.sleep(10)

    print("Searching for Restaurants...")
    best_rest = search.invoke(f"Best Restaurants in {destiny}")
    time.sleep(10)

    print("Searching for Activities...")
    best_act = search.invoke(f"Best Activities to do in {destiny}")
    time.sleep(10)

    print("Searching for Hotels...")
    best_hotels = search.invoke(f"Best hotels to do in {destiny}")
    time.sleep(10)

    print("Searching for Weather...")
    best_weather = search.invoke(f"What will be the weather forecasting in {destiny} from {start_date} to {end_date}")


    return {
        "attractions": best_att,
        "restaurants": best_rest,
        "activities": best_act,
        "hotels": best_hotels,
        "weather": best_weather,
    }

def compile_itinerary(state:State):
    template="""
    <Role>
    You are a travel planner specialist. You will receive useful information about a travel and 
    your job is to compile and summarize everything for the user in a helpful way.
    </Role>

    <Destination>
    {destination}
    </Destination>

    <Attractions>
    {attractions}
    </Attractions>

    <Restaurants>
    {restaurants}
    </Restaurants>
    
    <Activities>
    {activities}
    </Activities>
    
    <Hotels>
    {hotels}
    </Hotels>
    
    <Weather Forecasting>
    {weather}
    </Weather Forecasting>
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["destination", "attractions", "restaurants", "activities","hotels","weather"]
    )

    chain = prompt | llm | StrOutputParser()

    response = chain.invoke({
        "destination":state["destiny"],
        "attractions":state["attractions"],
        "restaurants":state["restaurants"],
        "activities":state["activities"],
        "hotels":state["hotels"],
        "weather":state["weather"],
    })

    return {"final_itinerary":response}


In [81]:
graph = StateGraph(State)

graph.add_node("planner_agent",itinerary_tool)
graph.add_node("summary_agent",compile_itinerary)

graph.add_edge("planner_agent","summary_agent")

graph.set_entry_point("planner_agent")
graph.set_finish_point("summary_agent")


workflow = graph.compile()

# Second Workflow Chatbot - Use first as a tool

In [ ]:
planner_tool = workflow.as_tool()

tools = [planner_tool]
tool_node = ToolNode(tools)

llm_with_tools = llm.bind_tools(tools)

In [ ]:
def call_model_chat(state:MessagesState):   
    messages = state["messages"]

    prompt = ChatPromptTemplate.from_messages([
        ("system", """<Role>
                     You are a travel planner specialist.
                    </Role>
                    <Task>
                    Engange in a conversation with the user and find his destination, end/start date of the trip.
                    </Task>
                    <Instructions>
                      1. After you gather destination, end/start date of the trip, call your tool to get the full itinery of the trip.
                      2. Return the detailed itinerary for the user right after calling the tool.
                    </Instructions
           """),
        ("placeholder", "{messages}")
    ])

    chain = prompt | llm_with_tools

    response = chain.invoke({"messages":messages})
    
    return {
        "messages":[response]
    }



In [85]:
memory_chat = InMemorySaver()

In [86]:
chat_workflow = StateGraph(MessagesState)

chat_workflow.add_node("chat_model", call_model_chat)
chat_workflow.add_node("tools", tool_node)
chat_workflow.add_edge(START, "chat_model")
chat_workflow.add_edge("tools", "chat_model")


chat_workflow.add_conditional_edges(
    "chat_model",
    tools_condition
)

chat_graph = chat_workflow.compile(memory_chat)

In [87]:
message = HumanMessage(content="Hi I want a trip to Paris from 10th October 2025 to 30th October 2025")
state = {"messages": [message]}
config = {"configurable": {"thread_id": "3"}}

response = chat_graph.invoke(
    state, config=config
)

for ans in response["messages"]:
    ans.pretty_print()

================================ Human Message =================================

Hi I want a trip to Paris from 10th October 2025 to 30th October 2025
================================== Ai Message ==================================

Great choice! Paris is a wonderful destination. To help plan your trip, could you please share if you have any preferences or interests regarding attractions, activities, restaurants, weather, or hotels? This will help me tailor the itinerary to your liking.


In [88]:
message = HumanMessage(content="I dont have any preferences now")
state = {"messages": [message]}
config = {"configurable": {"thread_id": "3"}}

response = chat_graph.invoke(
    state, config=config
)

for ans in response["messages"]:
    ans.pretty_print()

Searching for Attractions...
Searching for Restaurants...
Searching for Activities...
Searching for Hotels...
Searching for Weather...
================================ Human Message =================================

Hi I want a trip to Paris from 10th October 2025 to 30th October 2025
================================== Ai Message ==================================

Great choice! Paris is a wonderful destination. To help plan your trip, could you please share if you have any preferences or interests regarding attractions, activities, restaurants, weather, or hotels? This will help me tailor the itinerary to your liking.
================================ Human Message =================================

I dont have any preferences now
================================== Ai Message ==================================
Tool Calls:
  LangGraph (call_MaNmvsP7lXfd0bOvn5P2aXQf)
 Call ID: call_MaNmvsP7lXfd0bOvn5P2aXQf
  Args:
    destiny: Paris
    start_date: 10th October 2025
    end_date: 30th O